In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn
# Import Scikit-Learn library for decision tree models
import sklearn 

from sklearn import datasets, linear_model, metrics, tree
from sklearn.utils import resample

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor


from sklearn.neighbors import KNeighborsRegressor

        
# Dataset splitting        
from sklearn.model_selection import train_test_split


from matplotlib import pyplot as plt


import os
import joblib

In [2]:
df = pd.read_csv("flo_dataset_scaled.csv")
df

,Unnamed: 0,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,other_1_amount_mmol,other_2_amount_mmol,total_volume_ml,...,x6_zinc stearate,x6_zinc undecylenate,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm
0,0,0.154575,-0.011188,-0.247025,-0.673379,-0.370637,-0.096002,0.865472,-0.146249,-0.660116,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,None,480,539
1,1,0.195883,0.280681,-0.439228,-0.658075,-0.370637,-0.096002,-0.675365,-0.146249,-0.653603,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.61,610,None
2,2,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,0.457150,-0.146249,-0.651010,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,560,595
3,3,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,0.457150,-0.146249,-0.651010,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,590,635
4,4,0.154575,-0.011188,-0.247025,-0.604513,-0.370637,-0.096002,0.865472,-0.146249,-0.596878,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,None,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,214,0.154575,-0.376755,0.713993,4.491519,-0.370637,-0.096002,-0.675365,-0.146249,4.075133,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,585,630
215,215,7.590031,7.504453,-0.439228,5.524499,-0.370637,-0.096002,-0.675365,-0.146249,5.023700,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,465,550
216,216,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,-0.675365,-0.146249,6.288457,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,465,None
217,217,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,-0.675365,-0.146249,6.288457,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,None,495,None


In [3]:
# Input for ML models
input_col = ['in_amount_mmol', 'p_amount_mmol', 'ligand_amount_mmol',
       'first_sol_amount_ml', 'second_sol_amount_ml', 'third_sol_amount_ml',
       'other_1_amount_mmol', 'other_2_amount_mmol', 'total_volume_ml',
       'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate',
       'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide',
       'x0_indium myristate', 'x0_indium oxalate', 'x0_indium palmitate',
       'x0_indium trifluoroacetate',
       'x0_indium tris(N,N-diisopropylacetamidinato)',
       'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas',
       'x1_phosphorus trichloride', 'x1_sodium phosphide',
       'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine',
       'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine',
       'x1_white phosphorus', 'x2_None', 'x2_dodecanethiol', 'x2_lauric acid',
       'x2_myristic acid', 'x2_oleic acid', 'x2_palmitic acid',
       'x2_stearic acid', 'x3_4-ethylpyridine', 'x3_None',
       'x3_dimethylformamide', 'x3_dodecylamine', 'x3_mesitylene',
       'x3_octadecene', 'x3_oleylamine', 'x3_trioctylamine',
       'x3_trioctylphosphine', 'x3_trioctylphosphine oxide', 'x4_None',
       'x4_dioctyl ether', 'x4_dioctylamine', 'x4_hexadecylamine',
       'x4_octylamine', 'x4_oleylamine', 'x4_toluene', 'x4_trioctylphosphine',
       'x4_trioctylphosphine oxide', 'x5_None', 'x5_trioctylphosphine',
       'x6_None', 'x6_acetic acid', 'x6_superhydride',
       'x6_tetrabutylammonium myristate', 'x6_zinc acetate', 'x6_zinc bromide',
       'x6_zinc chloride', 'x6_zinc iodide', 'x6_zinc octanoate',
       'x6_zinc oleate', 'x6_zinc stearate', 'x6_zinc undecylenate', 'x7_None',
       'x7_copper bromide', 'x7_oleic acid', 'x7_water', 'x7_zinc iodide']


In [4]:
# Load ML model for predicting absorbance

loaded_rf_ab = joblib.load('model_aug_abs_DecisionTree.joblib')

# Replace 'None' entries in 'abs_nm' column by predicted values.

a = 0
for index, row in df.iterrows():
    if row['abs_nm'] == 'None':
        X = df.loc[index, input_col].to_numpy()
        df.loc[index, 'abs_nm'] = loaded_rf_ab.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'Absorbane max (nm)' column are replaced.   
df.to_csv('dataset_augmented_abs.csv')
df_first_aug = pd.read_csv('dataset_augmented_abs.csv')

In [5]:
#Saves the row indexes to drop for absorbance modelling into a list
total_row_num = len(df_first_aug)
drop_list_emi =[]
for row_i in range(total_row_num):
    if df_first_aug['emission_nm'].values[row_i] == 'None':
        drop_list_emi.append(row_i)
    
len(drop_list_emi)

#Drops the appropriate rows
df_emission_scaled_encoded = df_first_aug.drop(drop_list_emi)
#Saves the data for absorbance modelling to CSV
df_emission_scaled_encoded.to_csv('dataset_scaled_emission.csv')

# Move to **'model flo emission'** notebook to optimize model for augmenting emission

In [6]:
# adding column 'abs_nm' to input
input_col.append('abs_nm')
input_col_2 = input_col.copy()

In [7]:
print(input_col_2)
input_col.remove('abs_nm')

['in_amount_mmol', 'p_amount_mmol', 'ligand_amount_mmol', 'first_sol_amount_ml', 'second_sol_amount_ml', 'third_sol_amount_ml', 'other_1_amount_mmol', 'other_2_amount_mmol', 'total_volume_ml', 'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate', 'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide', 'x0_indium myristate', 'x0_indium oxalate', 'x0_indium palmitate', 'x0_indium trifluoroacetate', 'x0_indium tris(N,N-diisopropylacetamidinato)', 'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas', 'x1_phosphorus trichloride', 'x1_sodium phosphide', 'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine', 'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine', 'x1_white phosphorus', 'x2_None', 'x2_dodecanethiol', 'x2_lauric acid', 'x2_myristic acid', 'x2_oleic acid', 'x2_palmitic acid', 'x2_stearic acid', 'x3_4-ethylpyridine', 'x3_None', 'x3_dimethylformamide', 'x3_dodecylamine', 'x3_mesitylene', 'x3_octadecene', 'x3_oleylamine', 'x3_t

In [8]:
# Now, the dataset only has 'None' values in the 'emission_nm' column

df_emi = pd.read_csv('dataset_augmented_abs.csv') 

# Load ML model for predicting emission

loaded_rf_emi = joblib.load('model_aug_emission_ExtraTrees.joblib')


# # Replace 'None' entries in 'emission_nm' column by predicted values.

a = 0
for index, row in df_emi.iterrows():
    if row['emission_nm'] == 'None':
        X = df_emi.loc[index, input_col_2].to_numpy()
        df_emi.loc[index, 'emission_nm'] = loaded_rf_emi.predict(X.reshape(1, -1))[0]
        a += 1

# # # Save the dataset where all 'None' values are replaced.

# # # Ready to use for other analysis.

df_emi.to_csv('dataset_augmented_abs_emission.csv')


## Aldjust dataset_augmented_abs_emission.csv to dataset_augmented_abs_emission_adjusted.csv

In [9]:
# adding column 'emission_nm' to input
input_col_2.append('emission_nm')
input_col_3 = input_col_2.copy()


In [10]:
print(input_col_3)
input_col_2.remove('emission_nm')

['in_amount_mmol', 'p_amount_mmol', 'ligand_amount_mmol', 'first_sol_amount_ml', 'second_sol_amount_ml', 'third_sol_amount_ml', 'other_1_amount_mmol', 'other_2_amount_mmol', 'total_volume_ml', 'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate', 'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide', 'x0_indium myristate', 'x0_indium oxalate', 'x0_indium palmitate', 'x0_indium trifluoroacetate', 'x0_indium tris(N,N-diisopropylacetamidinato)', 'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas', 'x1_phosphorus trichloride', 'x1_sodium phosphide', 'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine', 'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine', 'x1_white phosphorus', 'x2_None', 'x2_dodecanethiol', 'x2_lauric acid', 'x2_myristic acid', 'x2_oleic acid', 'x2_palmitic acid', 'x2_stearic acid', 'x3_4-ethylpyridine', 'x3_None', 'x3_dimethylformamide', 'x3_dodecylamine', 'x3_mesitylene', 'x3_octadecene', 'x3_oleylamine', 'x3_t

In [11]:
df_second_aug = pd.read_csv('dataset_augmented_abs_emission_adjusted.csv')
#Saves the row indexes to drop for diameter modelling into a list
total_row_num = len(df_second_aug)
drop_list_dia =[]
for row_i in range(total_row_num):
    if df_second_aug['diameter_nm'].values[row_i] == 'None':
        drop_list_dia.append(row_i)
    
len(drop_list_dia)

#Drops the appropriate rows
df_dia_scaled_encoded = df_second_aug.drop(drop_list_dia)
#Saves the data for absorbance modelling to CSV
df_dia_scaled_encoded.to_csv('dataset_scaled_diameter.csv')

# Move to **'model hao diameter'** notebook to optimize model for augmenting emission.


In [12]:
# Load ML model for predicting diameters

df_dia = pd.read_csv('dataset_augmented_abs_emission_adjusted.csv') 

loaded_rf_dia = joblib.load('model_aug_diameter_DecisionTree.joblib')

# Replace 'None' entries in 'diameter_nm' column by predicted values.

a = 0
for index, row in df_dia.iterrows():
    if row['diameter_nm'] == 'None':
        X = df_dia.loc[index, input_col_3].to_numpy()
        df_dia.loc[index, 'diameter_nm'] = loaded_rf_dia.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'diameter_nm' column are replaced.   
df_dia.to_csv('flo_dataset_augmented.csv')